In [85]:
import pandas as pd
from ydata_profiling import ProfileReport

# Diagnostic sur les données. Le fichier en entrée est où sauvegarder le rapport.
def diagnostic(donnees, titre, fichier):
    rapport = ProfileReport(donnees, title=titre, explorative=True, correlations={"auto": {"calculate": False}})
    rapport.to_file(fichier)

# Chargement des données
fichier = "netflix_titles.csv"
donnees = pd.read_csv(fichier, encoding='utf-8', encoding_errors='ignore')  # Chargement du fichier csv
print(f"Nombre de lignes : {donnees.shape[0]}; Nombre de colonnes : {donnees.shape[1]}")
print(f"Données chargées depuis le fichier \"{fichier}\".")

# Diagnostic sous format HTML
diagnostic(donnees, "Rapport de pré-nettoyage", "pré-nettoyage.html") # Diagnostic pré-nettoyage

Nombre de lignes : 8807; Nombre de colonnes : 12
Données chargées depuis le fichier "netflix_titles.csv".


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
# --------------------------------------------------------------------------------------------------------------------------------------
# Après analyse du fichier HTML:
# Valeurs manquantes:
#   director (29.9%) (2634): impossible à imputer
#   cast (9.4%) (825): impossible à imputer
#   country (9.4%) (831): impossible à imputer
#   date_added (0.1%) (10): impossible à imputer
#   rating: (< 0.1%) (4): impossible à imputer
#   duration (<0.1%) (3): impossible à imputer (mais pour ce nombre on peut chercher sur Wikipedia...)
#    ---> Les valeurs manquantes pour chacune de ces colonnes seront nettoyées selon l'analyse nécessaire. 
# Duplicats: aucun
# Affichage du type de données
def affiche_typedonnees(): # On aura besoin de réutiliser.
    print("-----------------Types de données :")
    print(donnees.dtypes)
affiche_typedonnees()

-----------------Types de données :
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


In [15]:
# --------------------------------------------------------------------------------------------------------------------------------------
# À part release_year, tous sont identifiés comme "object", donc il faudra normaliser.
# Affichage d'un aperçu des données
print(donnees.head())

  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2  September 24, 2021        

In [24]:
# --------------------------------------------------------------------------------------------------------------------------------------
# À la vue des données, seules date_aded peuvent être stockées dans une valeur qui n'est pas une string.
# On essaie avec convert_dtypes()
donnees = donnees.convert_dtypes()
affiche_typedonnees()

-----------------Types de données :
show_id         string[python]
type            string[python]
title           string[python]
director        string[python]
cast            string[python]
country         string[python]
date_added      string[python]
release_year             Int64
rating          string[python]
duration        string[python]
listed_in       string[python]
description     string[python]
dtype: object


In [25]:
# --------------------------------------------------------------------------------------------------------------------------------------
# Convertit la date en un format utilisable.
# Référence: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
donnees["date_added"] = pd.to_datetime(donnees["date_added"], format='%B %d, %Y', errors='coerce')
print(donnees.head())

  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water             <NA>   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans             <NA>   
4      s5  TV Show           Kota Factory             <NA>   

                                                cast        country  \
0                                               <NA>  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...           <NA>   
3                                               <NA>           <NA>   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

  date_added  release_year rating   duration  \
0 2021-09-25          2020  PG-13     90 min   
1 2021-09-24          2021  TV-MA  2 Seasons   
2 2021-09-24          2021  TV-MA   1 Season   
3 2021-0

In [35]:
# --------------------------------------------------------------------------------------------------------------------------------------
# 2a) Films vs séries : proportions, tendances par année.
# Pour ces colonnes, aucune valeur manquante n'est reportée, donc aucun pré-traitement n'est nécessaire.
# Aperçu des types afin de s'assurer qu'ils ne sont pas dupliqués (majuscules-minuscules)
# Aperçu également des dates de sortie, afin de s'assurer que les années sont plausibles.
types = donnees.groupby("type")["type"].count()
annees_sortie = donnees.groupby("release_year")["release_year"].count()
print(types)
print(annees_sortie)

type
Movie      6131
TV Show    2676
Name: type, dtype: int64
release_year
1925       1
1942       2
1943       3
1944       3
1945       4
        ... 
2017    1032
2018    1147
2019    1030
2020     953
2021     592
Name: release_year, Length: 74, dtype: Int64


In [36]:
# --------------------------------------------------------------------------------------------------------------------------------------
# Concaténation d'une colonne de pourcentage pour les types, puis affichage
pourcentages = types / donnees.shape[0] * 100
types = pd.concat([types.rename("count"),pourcentages.rename("percent")], axis=1)
print(types)

         count    percent
type                     
Movie     6131  69.615079
TV Show   2676  30.384921


In [82]:
# --------------------------------------------------------------------------------------------------------------------------------------
# Tendances par année
#tendances = donnees.groupby("release_year")["type"].agg({"type":"sum"})
tendances = donnees.groupby(["release_year","type"])["type"].count()
print(tendances)
# TODO, faire un tableau plus compréhensible...

release_year  type   
1925          TV Show      1
1942          Movie        2
1943          Movie        3
1944          Movie        3
1945          Movie        3
                        ... 
2019          TV Show    397
2020          Movie      517
              TV Show    436
2021          Movie      277
              TV Show    315
Name: type, Length: 119, dtype: int64
